Import des librairies
- pixiedust (Kernel Python/Scala)
- time

In [13]:
#import pixiedust
import time
import os
import tweepy
import json
import json_lines


Configuration du nom du fichier utilisé

In [14]:
%env TIME={time.time()}

%env OUTPUT_TWEET_TXT_FILENAME=output_tweet_{os.environ['TIME']}.txt
%env TWEET_JSON_FILENAME=input_tweet_{os.environ['TIME']}.json


env: TIME=1521105739.7455528
env: OUTPUT_TWEET_TXT_FILENAME=output_tweet_1521105739.7455528.txt
env: TWEET_JSON_FILENAME=input_tweet_1521105739.7455528.json


In [15]:
#Variables that contains the user credentials to access Twitter API
API_key = json.load(open("API_key.txt", "r"))

auth = tweepy.OAuthHandler(API_key["consumer_key"], API_key["consumer_secret"])
auth.set_access_token(API_key["access_token"], API_key["access_token_secret"])
api = tweepy.API(auth)

keyword = "#Trump"
tweets = tweepy.Cursor(api.search, q=keyword).items(10)
json_tweets = [json.dumps(tweet._json) for tweet in tweets]
json_str = "\n\r".join(json_tweets)

filename = "tweets-database/"+os.environ['TWEET_JSON_FILENAME']
with open(filename, "w") as f:
    f.write(json_str)

In [16]:
%%bash
cd Algo1
cp trump.json ../tweets-database/$TWEET_JSON_FILENAME

Mise à jour de l'algorithme sur le Datacenter

In [17]:
%%bash
cd Algo1
sbt "push"

[info] Loading settings from plugins.sbt ...
[info] Loading project definition from /Users/Augustin/Documents/Cours/Centrale/Cours/S10/SparkSenti/SparkSenti/Algo1/project
[info] Loading settings from build.sbt ...
[info] Set current project to SparkSenti (in build file:/Users/Augustin/Documents/Cours/Centrale/Cours/S10/SparkSenti/SparkSenti/Algo1/)
[info] Exclude scala-library-2.11.8.jar from the package
[info] Packaging /Users/Augustin/Documents/Cours/Centrale/Cours/S10/SparkSenti/SparkSenti/Algo1/target/scala-2.11/sparksenti_2.11-0.1.jar ...
[info] Done packaging.
[info] [SparkSenti] Creating a distributable package in /Users/Augustin/Documents/Cours/Centrale/Cours/S10/SparkSenti/SparkSenti/Algo1/target/pack
[info] [SparkSenti] Copying libraries to /Users/Augustin/Documents/Cours/Centrale/Cours/S10/SparkSenti/SparkSenti/Algo1/target/pack/lib
[info] [SparkSenti] project jars:
[info] /Users/Augustin/Documents/Cours/Centrale/Cours/S10/SparkSenti/SparkSenti/Algo1/target/scala-2.11/sparks

Envoie des tweets collectés sur le datacenter

In [18]:
%%bash
cp tweets-database/$TWEET_JSON_FILENAME Algo1/$TWEET_JSON_FILENAME
cd Algo1
sbt "put $TWEET_JSON_FILENAME"

[info] Loading settings from plugins.sbt ...
[info] Loading project definition from /Users/Augustin/Documents/Cours/Centrale/Cours/S10/SparkSenti/SparkSenti/Algo1/project
[info] Loading settings from build.sbt ...
[info] Set current project to SparkSenti (in build file:/Users/Augustin/Documents/Cours/Centrale/Cours/S10/SparkSenti/SparkSenti/Algo1/)
spawn scp input_tweet_1521105739.7455528.json durivault1@156.18.90.100:.
durivault1@156.18.90.100's password: 
input_tweet_1521105739.7455528.json           100%  577KB   7.0MB/s   00:00    
spawn ssh durivault1@156.18.90.100
durivault1@156.18.90.100's password: 

Welcome to Bright release         7.3

                                                        Based on CentOS Linux 7
                                                                    ID: #000002

Use the following commands to adjust your environment:

'module avail'            - show available modules
'module add <module>'     - adds a module to your environment for this sessio

Execution de l'analyse de sentiments des tweets colléctés sur le datacenter

In [ ]:
%%bash
cd Algo1
rm -rf $TWEET_JSON_FILENAME
sbt "submit $TWEET_JSON_FILENAME"

Récupération du resultat de l'analyse : fichier text (id_tweet sentiment)

In [12]:
%%bash
cd Algo1
sbt "getOutput 'defaultoutput.txt'"
cp defaultoutput.txt ../tweets-database/$OUTPUT_TWEET_TXT_FILENAME
rm -rf defaultoutput.txt

[info] Loading settings from plugins.sbt ...
[info] Loading project definition from /Users/Augustin/Documents/Cours/Centrale/Cours/S10/SparkSenti/SparkSenti/Algo1/project
[info] Loading settings from build.sbt ...
[info] Set current project to SparkSenti (in build file:/Users/Augustin/Documents/Cours/Centrale/Cours/S10/SparkSenti/SparkSenti/Algo1/)
[error] java.io.FileNotFoundException: server.conf (No such file or directory)
[error] 	at java.io.FileInputStream.open(Native Method)
[error] 	at java.io.FileInputStream.<init>(FileInputStream.java:138)
[error] 	at scala.io.Source$.fromFile(Source.scala:91)
[error] 	at scala.io.Source$.fromFile(Source.scala:76)
[error] 	at scala.io.Source$.fromFile(Source.scala:54)
[error] 	at $e0d6e860966a7a8c1d4c$.getCredentials(build.sbt:33)
[error] 	at $fe7d0f3756d3a4a2351f$.$anonfun$$sbtdef$3(/Users/Augustin/Documents/Cours/Centrale/Cours/S10/SparkSenti/SparkSenti/Algo1/build.sbt:67)
[error] 	at $fe7d0f3756d3a4a2351f$$$Lambda$2397/407567224.apply$mcV$s

cp: defaultoutput.txt: No such file or directory


Pour test de viz (evite de lancer le workflow complet)

In [ ]:
%env OUTPUT_TWEET_TXT_FILENAME=output_test.txt
%env TWEET_JSON_FILENAME=input_test.json

Mapping des données pour Viz 

In [ ]:
file_output=open("tweets-database/"+os.environ['OUTPUT_TWEET_TXT_FILENAME'],"r", encoding="utf8");
file_tweet=open("tweets-database/"+os.environ['TWEET_JSON_FILENAME'],"r", encoding="utf8");


def parserByTweet(file_output,file_tweet):
    fileText = file_output.read().replace("Négatif","Negatif")
    fileTweets = fileText.split("\n\n")[1:]
    
    tweetList = []
    
    
    for tweet in fileTweets[:-1]:
        data=tweet.split("\n")
        tweet_text = data[0][7:]
        result = data[1][7:]
        tweet_id = data[2][9:]
        for tweet_json in json_lines.reader(file_tweet):
            if int(tweet_id)==int(tweet_json["id"]):
                new={}
                new["result"]=result
                new["tweet"]=tweet_json
                tweetList.append(new)  
    


    return(tweetList)
    
data_to_viz=parserByTweet(file_output,file_tweet)
print(data_to_viz)